In [1]:
from aeon.distances import (
    euclidean_distance as aeon_euclidean_distance,
    dtw_distance as aeon_dtw_distance,
    adtw_distance as aeon_adtw_distance,
    ddtw_distance as aeon_ddtw_distance,
    erp_distance as aeon_erp_distance,
    edr_distance as aeon_edr_distance,
    lcss_distance as aeon_lcss_distance,
    manhattan_distance as aeon_manhattan_distance,
    minkowski_distance as aeon_minkowski_distance,
    msm_distance as aeon_msm_distance,
    sbd_distance as aeon_sbd_distance,
    shape_dtw_distance as aeon_shape_dtw_distance,
    squared_distance as aeon_squared_distance,
    twe_distance as aeon_twe_distance,
    wddtw_distance as aeon_wddtw_distance,
    wdtw_distance as aeon_wdtw_distance,
)

In [2]:
import numpy as np
import math

In [3]:
from numba import njit

In [4]:
from aeon.distances import euclidean_distance as aeon_ed, dtw_distance as aeon_dtw

In [5]:
import numpy as np

In [304]:
# Sample sequences
length = 200
np.random.seed(18)
q = np.sin(np.linspace(0, 4 * np.pi, length))
c = q + np.random.normal(0, 0.4, size=length)
len_q = len(q)
len_c = len(c)
print(f"Length of q: {len_q}, Length of c: {len_c}")

Length of q: 200, Length of c: 200


In [ ]:
# @njit
# def nearest_neighbor_interpolation_old(C, L):
#     """
#     Implements exact nearest neighbor interpolation as defined by:
#         CL[j] = C[ceil(j * k / L)] for j in [1, L]

#     Args:
#         C (list or np.array): Original sequence (length k)
#         L (int): Desired rescaled length

#     Returns:
#         np.array: Rescaled sequence of length L
#     """
#     C = np.asarray(C)
#     k = len(C)
#     indices = [
#         int(np.ceil(j * k / L)) - 1 for j in range(1, L + 1)
#     ]  # 1-based to 0-based
#     return C[indices]

In [ ]:
from numba import njit
import numpy as np


@njit
def nearest_neighbor_interpolation(C, L):
    """
    Implements exact nearest neighbor interpolation:
        CL[j] = C[ceil(j * k / L)] for j in [1, L]
    """
    k = len(C)
    result = np.empty(L, dtype=C.dtype)
    for j in range(L):
        idx = int(np.ceil((j + 1) * k / L)) - 1  # 1-based to 0-based
        result[j] = C[idx]
    return result

In [9]:
# %time
import time

start_time = time.time()
rescaled = nearest_neighbor_interpolation(q, 350)
end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

Execution time: 0.000068 seconds


In [55]:
@njit
def usdtw_prime(Q, C, L, r, dist_method):
    # Q_scaled = nearest_neighbor_interpolation_njit(Q, len(Q), L)
    # C_scaled = nearest_neighbor_interpolation_njit(C, len(C), L)
    Q_scaled = nearest_neighbor_interpolation(Q, L)
    C_scaled = nearest_neighbor_interpolation(C, L)
    if dist_method == 0:
        # return aeon_euclidean_distance(Q_scaled, C_scaled) ** 2
        return aeon_squared_distance(Q_scaled, C_scaled)
    elif dist_method == 1:
        return aeon_dtw_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 2:
        return aeon_adtw_distance(Q_scaled, C_scaled, window=r)
        # return aeon_shape_dtw_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 3:
        return aeon_ddtw_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 4:
        return aeon_erp_distance(Q_scaled, C_scaled, window=r)
        # return aeon_wdtw_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 5:
        return aeon_edr_distance(Q_scaled, C_scaled, window=r)
        # return aeon_wddtw_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 6:
        return aeon_lcss_distance(Q_scaled, C_scaled, window=r)
        # return aeon_adtw_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 7:
        return aeon_manhattan_distance(Q_scaled, C_scaled)
        # return aeon_erp_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 8:
        return aeon_minkowski_distance(Q_scaled, C_scaled)
        # return aeon_edr_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 9:
        return aeon_msm_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 10:
        return aeon_sbd_distance(Q_scaled, C_scaled)
        # return aeon_twe_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 11:
        return aeon_shape_dtw_distance(Q_scaled, C_scaled, window=r)
        # return aeon_lcss_distance(Q_scaled, C_scaled, window=r)
    elif dist_method == 12:
        return aeon_squared_distance(Q_scaled, C_scaled)
        # return aeon_manhattan_distance(Q_scaled, C_scaled)
    elif dist_method == 13:
        return aeon_twe_distance(Q_scaled, C_scaled, window=r)
        # return aeon_minkowski_distance(Q_scaled, C_scaled)
    elif dist_method == 14:
        return aeon_wddtw_distance(Q_scaled, C_scaled, window=r)
        # return aeon_sbd_distance(Q_scaled, C_scaled)
    elif dist_method == 15:
        return aeon_wdtw_distance(Q_scaled, C_scaled, window=r)
    else:
        raise ValueError("Invalid distance method!")

In [47]:
usdtw_prime(q, c, 150, -1, 0)

506.42304820862256

In [127]:
@njit
def psdtw_prime_vanilla(Q, C, l, P, r, dist_method):
    # print("Using psdtw_prime_test")
    count_dist_calls = 0
    m = len(Q)
    n = len(C)
    assert m == n, "m should be equal to n"
    l_root = math.sqrt(l)
    L_avg = m / P
    L_min = max(1, int(math.ceil(L_avg / l_root)))
    L_max = min(int(math.floor(L_avg * l_root)), m)

    # Minimum cost to align the **first i** elements of Q (i.e., Q[:i]) and the **first j** elements of C (i.e., C[:j]) using **P** exactly segments
    # Each segment satisfies the length constraint
    D = np.full((m + 1, n + 1, P + 1), np.inf)
    D[0, 0, 0] = 0.0
    D_cut = np.full((m + 1, n + 1, P + 1, 2), -1, dtype=np.int64)

    for p in range(1, P + 1):
        L_acc_min = L_min * p  # p segments take at least "L_min" points
        L_acc_max = L_max * p

        for i in range(L_acc_min, min(L_acc_max, m) + 1):
            for L_Q in range(L_min, L_max + 1):
                i_prime = i - L_Q
                L_C_min = max(L_min, int(math.ceil(L_Q / l)))
                L_C_max = min(int(math.floor(L_Q * l)), L_max)

                for j in range(L_acc_min, min(L_acc_max, n) + 1):
                    for L_C in range(L_C_min, L_C_max + 1):
                        j_prime = j - L_C
                        D_cost = D[i_prime, j_prime, p - 1]
                        # Lower bounds
                        if np.isinf(D_cost):
                            continue
                        if D_cost > D[i][j][p]:  # best_so_far
                            continue
                        dist_cost = usdtw_prime(
                            Q[i_prime:i][::-1],
                            C[j_prime:j][::-1],
                            L=L_max,
                            r=r,
                            dist_method=dist_method,
                        )
                        count_dist_calls += 1
                        # D[i, j, p] = min(D[i, j, p], D_cost + dist_cost)
                        cur_cost = D_cost + dist_cost
                        if cur_cost < D[i, j, p]:
                            D[i, j, p] = cur_cost
                            D_cut[i, j, p, 0] = i_prime
                            D_cut[i, j, p, 1] = j_prime
    cuts = np.zeros((P, 4), dtype=np.int64)
    i, j, p = m, n, P
    while p > 0:
        i_prime = D_cut[i, j, p, 0]
        j_prime = D_cut[i, j, p, 1]
        cuts[p - 1, 0] = i_prime
        cuts[p - 1, 1] = i
        cuts[p - 1, 2] = j_prime
        cuts[p - 1, 3] = j
        i, j, p = i_prime, j_prime, p - 1
    return D[m, n, P], count_dist_calls, cuts

In [129]:
@njit(inline="always")
def lb_shen_prefix(Q, C, l, r):
    m = len(Q)
    dist_total = (Q[0] - C[0]) ** 2
    for j in range(1, m):
        start = int(max(0, np.ceil(j / l) - r))
        end = int(min(np.ceil(j * l) + r, m - 1))
        min_dist = (C[j] - Q[start]) ** 2
        for k in range(start + 1, end + 1):
            d = (C[j] - Q[k]) ** 2
            if d < min_dist:
                min_dist = d
        dist_total += min_dist
    return dist_total

In [218]:
@njit(inline="always")
def lb_shen_incremental(Q, C, l, r):
    m = len(Q)
    j_last = len(C) - 1
    start = int(max(0, np.ceil(j_last / l) - r))
    end = int(min(np.ceil(j_last * l) + r, m - 1))
    min_dist = (C[j_last] - Q[start]) ** 2
    for k in range(start + 1, end + 1):
        d = (C[j_last] - Q[k]) ** 2
        if d < min_dist:
            min_dist = d
    return min_dist

In [ ]:
@njit
def psdtw_prime_lb_shen(Q, C, l, P, r, dist_method):
    # print("Using psdtw_prime_test")
    count_dist_calls = 0
    m = len(Q)
    n = len(C)
    assert m == n, "m should be equal to n"
    l_root = math.sqrt(l)
    L_avg = m / P
    L_min = max(1, int(math.ceil(L_avg / l_root)))
    L_max = min(int(math.floor(L_avg * l_root)), m)

    # Minimum cost to align the **first i** elements of Q (i.e., Q[:i]) and the **first j** elements of C (i.e., C[:j]) using **P** exactly segments
    # Each segment satisfies the length constraint
    D = np.full((m + 1, n + 1, P + 1), np.inf)
    D[0, 0, 0] = 0.0
    D_cut = np.full((m + 1, n + 1, P + 1, 2), -1, dtype=np.int64)

    for p in range(1, P + 1):
        L_acc_min = L_min * p  # p segments take at least "L_min" points
        L_acc_max = L_max * p

        for i in range(L_acc_min, min(L_acc_max, m) + 1):
            for L_Q in range(L_min, L_max + 1):
                i_prime = i - L_Q
                L_C_min = max(L_min, int(math.ceil(L_Q / l)))
                L_C_max = min(int(math.floor(L_Q * l)), L_max)

                for j in range(L_acc_min, min(L_acc_max, n) + 1):
                    lb = 0.0
                    for L_C in range(L_C_min, L_C_max + 1):
                        j_prime = j - L_C
                        D_cost = D[i_prime, j_prime, p - 1]
                        # Lower bounds
                        if np.isinf(D_cost):
                            continue
                        if D_cost > D[i][j][p]:  # best_so_far
                            continue
                        if L_C == L_C_min:
                            lb = lb_shen_prefix(
                                Q[i_prime:i][::-1], C[j_prime:j][::-1], l=l, r=r
                            )
                        else:
                            lb += lb_shen_incremental(
                                Q[i_prime:i][::-1], C[j_prime:j][::-1], l=l, r=r
                            )
                            # lb = lb_shen_prefix(Q[i_prime:i][::-1], C[j_prime:j][::-1], l=l, r=r)
                        if D_cost + lb > D[i][j][p]:  # best_so_far
                            continue
                        dist_cost = usdtw_prime(
                            Q[i_prime:i][::-1],
                            C[j_prime:j][::-1],
                            L=L_max,
                            r=r,
                            dist_method=dist_method,
                        )
                        count_dist_calls += 1
                        # D[i, j, p] = min(D[i, j, p], D_cost + dist_cost)
                        cur_cost = D_cost + dist_cost
                        if cur_cost < D[i, j, p]:
                            D[i, j, p] = cur_cost
                            D_cut[i, j, p, 0] = i_prime
                            D_cut[i, j, p, 1] = j_prime
    cuts = np.zeros((P, 4), dtype=np.int64)
    i, j, p = m, n, P
    while p > 0:
        i_prime = D_cut[i, j, p, 0]
        j_prime = D_cut[i, j, p, 1]
        cuts[p - 1, 0] = i_prime
        cuts[p - 1, 1] = i
        cuts[p - 1, 2] = j_prime
        cuts[p - 1, 3] = j
        i, j, p = i_prime, j_prime, p - 1
    return D[m, n, P], count_dist_calls, cuts

In [305]:
import time

start_time = time.time()
print(psdtw_prime_vanilla(q, c, 1.5, 3, 0.1, 1))
end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

(29.64491184053319, 895418, array([[  0,  75,   0,  68],
       [ 75, 144,  68, 145],
       [144, 200, 145, 200]]))
Execution time: 7.649931 seconds


(67.9680839466124, 15402290, array([[  0, 153,   0, 150],
       [153, 263, 150, 265],
       [263, 400, 265, 400]]))
Execution time: 587.815522 seconds

In [306]:
start_time = time.time()
print(psdtw_prime_lb_shen(q, c, 1.5, 3, 0.1, 1))
end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

(29.64491184053319, 804282, array([[  0,  75,   0,  68],
       [ 75, 144,  68, 145],
       [144, 200, 145, 200]]))
Execution time: 7.079806 seconds


(33.60247725157748, 1593779, array([[  0,  47,   0,  48],
       [ 47, 107,  48, 107],
       [107, 150, 107, 150]]))
Execution time: 2.533447 seconds

In [170]:
start_time = time.time()
lb_shen_prefix(q[::-1], c[::-1], 2.0, 0.1)
end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

Execution time: 0.000183 seconds


In [172]:
start_time = time.time()
usdtw_prime(q, c, len(q), 0.1, 1)
end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

Execution time: 0.001307 seconds


In [192]:
# load datasets
# A neat way to load the dataset, but more complicated to use
# data = np.load("../data_intermediate/GunPoint_preprocessed_P_3_l_2.0_len_150.npz")
# data_dict = {key: data[key] for key in data.files}

# A old way to load the dataset
data = np.load(
    "../data_intermediate/GunPoint_ps_P_3_l_2.0_len_150.npz",
    allow_pickle=True,
)
X_train_scaled = data["X_train_scaled"]
X_train_ps = data["X_train_ps"]
X_train_ps_noise = data["X_train_ps_noise"]
y_train = data["y_train"]
X_test_scaled = data["X_test_scaled"]
X_test_ps = data["X_test_ps"]
X_test_ps_noise = data["X_test_ps_noise"]
y_test = data["y_test"]
X_train_cuts = data["X_train_cuts"].tolist()
X_train_ps_cuts = data["X_train_ps_cuts"].tolist()
X_test_cuts = data["X_test_cuts"].tolist()
X_test_ps_cuts = data["X_test_ps_cuts"].tolist()

from aeon.utils.numba.general import z_normalise_series_2d

X_train_scaled_norm = z_normalise_series_2d(X_train_scaled)
X_train_ps_norm = z_normalise_series_2d(X_train_ps)
X_train_ps_noise_norm = z_normalise_series_2d(X_train_ps_noise)

# *** Change here 1 ***
# Query set
# query_set = X_train_ps
query_set = X_train_ps_norm
# query_set = X_train_ps_noise
# query_set = X_train_ps_noise_norm


# Target set
# target_set = X_train_scaled
target_set = X_train_scaled_norm
if len(query_set) != len(target_set):
    raise ValueError("query_set and target_set have different sizes!")

In [201]:
print(psdtw_prime_vanilla(query_set[0], target_set[0], 2, 3, 0, 0))

(3.3654395029420554, 1807925, array([[  0,  41,   0,  48],
       [ 41, 110,  48, 108],
       [110, 150, 108, 150]]))


In [194]:
print(psdtw_prime_lb_shen(query_set[0], target_set[0], 2, 3, 0, 0))

(3.3654395029420554, 1438897, array([[  0,  41,   0,  48],
       [ 41, 110,  48, 108],
       [110, 150, 108, 150]]))


In [295]:
for i in range(0, 1):
    for j in range(0, 50):
        print(j)
        if np.isclose(
            psdtw_prime_vanilla(query_set[i], target_set[j], 2, 3, 0.1, 1)[0],
            psdtw_prime_lb_shen(query_set[i], target_set[j], 2, 3, 0.1, 1)[0],
        ):
            continue
        else:
            print(f"Values differ at i={i}, j={j}")
# Values differ at i=0, j=3

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [199]:
len(target_set)

50

---
---
---

In [9]:
@njit
def euclidean_distance(x, y):
    """
    Calculate Euclidean distance between two sequences using numba.

    Parameters:
    x, y: array-like sequences of the same length

    Returns:
    float: Euclidean distance
    """
    if len(x) != len(y):
        raise ValueError("Sequences must have the same length")

    distance = 0.0
    for i in range(len(x)):
        distance += (x[i] - y[i]) ** 2

    return distance

In [65]:
%%timeit
euclidean_distance(q, c)

457 ns ± 1.31 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
%%timeit
aeon_ed(q, c)

242 ns ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
euclidean_distance(q, c), aeon_ed(q, c)

(32.60165605255884, 5.70978599008394)

In [129]:
@njit
def dtw(Q, C, window=None):
    m, n = len(Q), len(C)

    # Convert fractional window to absolute value
    if window is None:
        r = max(m, n)  # No constraint if window is None
    else:
        r = int(window * max(m, n))  # Convert fraction to absolute value

    if abs(n - m) > r:
        raise ValueError("abs(n-m) > r!")

    D = np.full((n + 1, m + 1), np.inf)
    D[0, 0] = 0.0
    for i in range(1, m + 1):
        for j in range(max(1, i - r), min(n, i + r) + 1):
            cost = (Q[i - 1] - C[j - 1]) ** 2
            D[i, j] = cost + min(D[i - 1, j], D[i, j - 1], D[i - 1, j - 1])
    return D[m, n]

In [130]:
%%timeit
dtw(q, c, window=0.1)

112 μs ± 89.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [15]:
%%timeit
aeon_dtw(q, c, window=0.1)

63.9 μs ± 1.8 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
dtw(q, c, window=0.03), aeon_dtw(q, c, window=0.03)

(27.35818127369896, 27.35818127369896)

In [134]:
@njit
def usdtw(Q, C, l, L, r, dist_method=0):
    m, n = len(Q), len(C)
    best_so_far = np.inf
    best_k = -1
    for k in range(math.ceil(m / l), min(math.floor(l * m), n) + 1):
        C_prefix = C[:k]
        dist = usdtw_prime(Q, C_prefix, L, r, dist_method)
        if dist < best_so_far:
            best_so_far = dist
            best_k = k
    return best_so_far, best_k

---

In [125]:
len(q)

200

In [ ]:
np.allclose(
    nearest_neighbor_interpolation_round_njit(q[::-1], 200, 250)[::-1],
    nearest_neighbor_interpolation_round_njit(q, 200, 250),
)

False

In [127]:
nearest_neighbor_interpolation_round_njit(q[::-1], 200, 250)[::-1]

array([ 0.00000000e+00,  6.31056313e-02,  1.25959705e-01,  1.25959705e-01,
        1.88311666e-01,  2.49912962e-01,  3.10518032e-01,  3.69885285e-01,
        3.69885285e-01,  4.27778068e-01,  4.83965601e-01,  5.38223906e-01,
        5.90336692e-01,  5.90336692e-01,  6.40096223e-01,  6.87304143e-01,
        7.31772266e-01,  7.73323331e-01,  7.73323331e-01,  8.11791702e-01,
        8.47024034e-01,  8.78879879e-01,  9.07232251e-01,  9.07232251e-01,
        9.31968129e-01,  9.52988909e-01,  9.70210796e-01,  9.83565137e-01,
        9.83565137e-01,  9.92998700e-01,  9.98473878e-01,  9.99968847e-01,
        9.97477646e-01,  9.97477646e-01,  9.91010207e-01,  9.80592311e-01,
        9.66265486e-01,  9.48086844e-01,  9.48086844e-01,  9.26128849e-01,
        9.00479032e-01,  8.71239643e-01,  8.38527236e-01,  8.38527236e-01,
        8.02472214e-01,  7.63218303e-01,  7.20921979e-01,  6.75751849e-01,
        6.75751849e-01,  6.27887973e-01,  5.77521152e-01,  5.24852163e-01,
        4.70090958e-01,  

In [144]:
C_R = C[::-1]
C_R_L = nearest_neighbor_interpolation(C_R, k=5, L=8)
print(C_R_L)

[50 40 40 30 20 20 10 10]


In [146]:
def nearest_neighbor_interp_symmetric(C, k, L):
    Ck = np.asarray(C[:k])
    idx = np.round(np.arange(1, L + 1) * k / L).astype(int)
    idx = np.clip(idx - 1, 0, k - 1)  # 1-based to 0-based
    return Ck[idx]

In [ ]:
C_L = nearest_neighbor_interp_symmetric(C, k=5, L=8)
print(C_L[::-1])

[50 40 40 30 20 20 10 10]


In [151]:
C_R = C[::-1]
C_R_L = nearest_neighbor_interp_symmetric(C_R, k=5, L=8)
print(C_R_L)

[50 50 40 40 30 20 20 10]


---

In [ ]:
import numpy as np


def nn_interp_symmetric(T, k, L):
    T = np.asarray(T)
    if L == 1:
        return T[:1].copy()
    s = (np.arange(L) * (k - 1)) / (L - 1)  # 0..k-1, inclusive
    # Use half-up rounding to avoid banker's-rounding surprises
    idx = np.floor(s + 0.5).astype(np.int64)
    idx = np.clip(idx, 0, k - 1)
    return T[:k][idx]

In [ ]:
T = np.array([10, 20, 30, 40, 50])
k, L = 5, 8

A = nn_interp_symmetric(T, k, L)[::-1]  # Rev(Interp(T))
B = nn_interp_symmetric(T[::-1], k, L)  # Interp(Rev(T))
assert np.array_equal(A, B)  # passes